<a href="https://colab.research.google.com/github/Kisara00555/1st-Project/blob/main/Computational_Structural_Data_Acquisition_from_PDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Computational Structural Data Acquisition from the Protein Data Bank (PDB) Workshop


End-to-end workflow in one notebook:

1. Search and select a **PDB entry** (optional)
2. Download the **entry coordinates** from RCSB
3. Identify ligands and fetch **CCD/chemistry** (SMILES) for ligands
4. Visualize protein + ligand with **py3Dmol**
5. Fix the structure (missing residues/atoms, add H) using **PDBFixer** and save to `/Users/samithir/Downloads/...`

**Tip:** If you already know your PDB ID, you can skip the search section and jump to the download section.

## 1) Install & import (pip)

- `openmm` is available via `pip`.
- `pdbfixer` is installed from the official OpenMM GitHub repo via `pip`.

Run this cell once per environment.

In [2]:
%pip -q install requests pandas matplotlib tqdm py3Dmol
%pip -q install openmm
%pip -q install rdkit
%pip -q install "git+https://github.com/openmm/pdbfixer.git"

import json
import time
from dataclasses import dataclass
from pathlib import Path
from typing import Any, Dict, List, Optional, Iterable, Tuple

import requests
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

from IPython.display import display, Markdown
import py3Dmol

from pdbfixer import PDBFixer
import openmm.app as app

# Optional: RDKit (for SMILES->3D + descriptors). If not available, the notebook will still run.
try:
    from rdkit import Chem
    from rdkit.Chem import Descriptors, AllChem
    RDKit_OK = True
except Exception:
    Chem = None
    Descriptors = None
    AllChem = None
    RDKit_OK = False

plt.rcParams["figure.figsize"] = (6, 4)

session = requests.Session()
session.headers.update({"User-Agent": "pdb-workflow-notebook/1.0"})


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 50.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


## 2) Endpoints + helpers

In [3]:
SEARCH_URL = "https://search.rcsb.org/rcsbsearch/v2/query"
DATA_CORE_URL = "https://data.rcsb.org/rest/v1/core"
FILES_URL = "https://files.rcsb.org/download"
LIGANDS_URL = "https://files.rcsb.org/ligands/download"
MODELS_URL = "https://models.rcsb.org/v1"

def request_json(url: str, *, timeout: int = 60) -> Dict[str, Any]:
    r = session.get(url, timeout=timeout)
    r.raise_for_status()
    return r.json()

def download_text(url: str, *, timeout: int = 60) -> str:
    r = session.get(url, timeout=timeout)
    r.raise_for_status()
    return r.text

def download_file(url: str, outpath: Path, *, timeout: int = 60) -> Path:
    r = session.get(url, timeout=timeout)
    r.raise_for_status()
    outpath.write_bytes(r.content)
    return outpath


## 3) Search for entries by UniProt accession

If you already know your `PDB_ID`, skip this section.

Enter a UniProt accession, run the cell, then set `PDB_ID` below to one of the hits.

In [4]:
import requests
from typing import List

SEARCH_URL = "https://search.rcsb.org/rcsbsearch/v2/query"
session = requests.Session()

def search_entries_by_uniprot(uniprot_acc: str, rows: int = 25, start: int = 0) -> List[str]:
    uniprot_acc = uniprot_acc.strip()
    if not uniprot_acc:
        return []

    query = {
        "query": {
            "type": "group",
            "logical_operator": "and",
            "nodes": [
                {
                    "type": "terminal",
                    "service": "text",
                    "parameters": {
                        "attribute": "rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_accession",
                        "operator": "exact_match",
                        "value": uniprot_acc,
                    },
                },
                {
                    "type": "terminal",
                    "service": "text",
                    "parameters": {
                        "attribute": "rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_name",
                        "operator": "exact_match",
                        "value": "UniProt",
                    },
                },
            ],
        },
        "return_type": "entry",
        "request_options": {
            "results_content_type": ["experimental"],   # v2 field
            "paginate": {"start": start, "rows": rows}, # v2 field
        },
    }

    r = session.post(SEARCH_URL, json=query, timeout=60)
    if not r.ok:
        # show the server message to debug quickly
        raise RuntimeError(f"HTTP {r.status_code}: {r.text}")

    data = r.json()
    return [item["identifier"] for item in data.get("result_set", [])]

UNIPROT_ACC = "P69905"
hits = search_entries_by_uniprot(UNIPROT_ACC, rows=25, start=0)
print("Hits:", len(hits))
print(hits)



Hits: 25
['1A00', '1A01', '1A0U', '1A0Z', '1A3N', '1A3O', '1A9W', '1ABW', '1ABY', '1AJ9', '1B86', '1BAB', '1BBB', '1BIJ', '1BUW', '1BZ0', '1BZ1', '1BZZ', '1C7B', '1C7C', '1C7D', '1CLS', '1CMY', '1COH', '1DKE']


## 4) User settings

Set the PDB entry and ligand code you want to keep.

Output is written to: `/Users/samithir/Downloads/<PDB_ID>_workflow/`

In [8]:
PDB_ID = "4HHB"   # <-- set this
LIG_ID = "HEM"    # <-- set this (CCD 3-letter code; e.g., NO3/EDO/KY9)

PH = 7.0
REMOVE_WATERS = True
BUILD_MISSING_RESIDUES = True   # set False to avoid loop building

OUTDIR = Path("/Users/ASUS/Downloads") / f"{PDB_ID}_workflow"
OUTDIR.mkdir(parents=True, exist_ok=True)

RAW_PDB_PATH = OUTDIR / f"{PDB_ID}.pdb"
FIXED_PDB_PATH = OUTDIR / f"{PDB_ID}_fixed.pdb"
LIGAND_ONLY_PDB_PATH = OUTDIR / f"{PDB_ID}_{LIG_ID}_ligand_only.pdb"

print("OUTDIR:", OUTDIR)

OUTDIR: /Users/ASUS/Downloads/4HHB_workflow


## 5) Download the PDB entry coordinates

This tries legacy `.pdb` first; if not available, it falls back to mmCIF.

In [11]:
print(f"Listing contents of {OUTDIR}:")
!ls -lh {OUTDIR}

Listing contents of /Users/ASUS/Downloads/4HHB_workflow:
total 464K
-rw-r--r-- 1 root root 463K Dec 14 03:36 4HHB.pdb


In [13]:
from google.colab import files

# Ensure the file path is correct based on the OUTDIR variable
file_to_download = RAW_PDB_PATH # Changed to download the raw PDB file as requested
# If you meant the raw PDB file, use RAW_PDB_PATH instead:
# file_to_download = RAW_PDB_PATH # This variable points to /Users/ASUS/Downloads/4HHB_workflow/4HHB.pdb

if file_to_download.exists():
    files.download(str(file_to_download))
    print(f"Downloaded {file_to_download.name} to your local machine.")
else:
    print(f"File not found: {file_to_download}. Please check the path.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Downloaded 4HHB.pdb to your local machine.


In [9]:
pdb_url = f"{FILES_URL}/{PDB_ID}.pdb"
try:
    download_file(pdb_url, RAW_PDB_PATH)
    print("Downloaded legacy PDB:", RAW_PDB_PATH)
    COORD_FORMAT = "pdb"
except Exception as e:
    cif_url = f"{FILES_URL}/{PDB_ID}.cif"
    cif_path = OUTDIR / f"{PDB_ID}.cif"
    download_file(cif_url, cif_path)
    print("Downloaded mmCIF:", cif_path)
    COORD_FORMAT = "cif"
    print("NOTE: The PDBFixer part of this notebook expects a PDB file. If you only have mmCIF, consider converting mmCIF->PDB with a structure tool.")


Downloaded legacy PDB: /Users/ASUS/Downloads/4HHB_workflow/4HHB.pdb


## 6) Quick ligand list from HETATM records

This is a quick way to inspect ligand codes present in the downloaded legacy PDB.

In [14]:
if COORD_FORMAT != "pdb":
    print("Skipping HETATM parsing because we did not download a legacy PDB.")
else:
    pdb_lines = RAW_PDB_PATH.read_text(errors="ignore").splitlines()
    het_resn = sorted({line[17:20].strip() for line in pdb_lines if line.startswith("HETATM")})
    print("HETATM resnames:", het_resn)


HETATM resnames: ['HEM', 'HOH', 'PO4']


## 7) Fetch CCD (Chemical Component) info for ligand(s)

We use the RCSB Data API core `chemcomp` object to pull metadata and (when available) SMILES.

In [15]:
_chemcomp_cache: Dict[str, Dict[str, Any]] = {}

def fetch_chemcomp(comp_id: str) -> Dict[str, Any]:
    comp_id = comp_id.upper()
    if comp_id in _chemcomp_cache:
        return _chemcomp_cache[comp_id]
    d = request_json(f"{DATA_CORE_URL}/chemcomp/{comp_id}")
    _chemcomp_cache[comp_id] = d
    return d

def chemcomp_to_smiles(d: Dict[str, Any]) -> Optional[str]:
    desc = d.get("rcsb_chem_comp_descriptor") or {}
    if isinstance(desc, dict) and desc.get("smiles"):
        return desc["smiles"]
    descs = d.get("pdbx_chem_comp_descriptor") or []
    if isinstance(descs, list):
        for item in descs:
            if (item.get("type") == "SMILES") and item.get("descriptor"):
                return item["descriptor"]
    return None

# You can expand this list if you want multiple ligands
lig_ids = [LIG_ID.upper()]

chem_records = []
for cid in lig_ids:
    try:
        d = fetch_chemcomp(cid)
        smiles = chemcomp_to_smiles(d)
        chem_records.append({
            "comp_id": cid,
            "name": (d.get("chem_comp") or {}).get("name"),
            "formula": (d.get("chem_comp") or {}).get("formula"),
            "formula_weight": (d.get("chem_comp") or {}).get("formula_weight"),
            "smiles": smiles,
        })
    except Exception as e:
        chem_records.append({"comp_id": cid, "name": None, "formula": None, "formula_weight": None, "smiles": None, "error": str(e)})

chem_df = pd.DataFrame(chem_records)
display(chem_df)


,comp_id,name,formula,formula_weight,smiles
0,HEM,PROTOPORPHYRIN IX CONTAINING FE,C34 H32 Fe N4 O4,616.487,Cc1c2n3c(c1CCC(=O)O)C=C4C(=C(C5=[N]4[Fe]36[N]7...


## 8) Compute RDKit descriptors (if RDKit is available)

In [16]:
if not RDKit_OK:
    print("RDKit not available; skipping descriptor calculation.")
else:
    def smiles_to_mol(smiles: Optional[str]):
        if not smiles:
            return None
        try:
            return Chem.MolFromSmiles(smiles)
        except Exception:
            return None

    chem_df = chem_df.copy()
    chem_df["mol"] = chem_df["smiles"].apply(smiles_to_mol)
    chem_df["mw"] = chem_df["mol"].apply(lambda m: Descriptors.MolWt(m) if m else None)
    chem_df["logp"] = chem_df["mol"].apply(lambda m: Descriptors.MolLogP(m) if m else None)
    chem_df["hbd"] = chem_df["mol"].apply(lambda m: Descriptors.NumHDonors(m) if m else None)
    chem_df["hba"] = chem_df["mol"].apply(lambda m: Descriptors.NumHAcceptors(m) if m else None)

    display(chem_df[["comp_id","name","formula","smiles","mw","logp","hbd","hba"]])


,comp_id,name,formula,smiles,mw,logp,hbd,hba
0,HEM,PROTOPORPHYRIN IX CONTAINING FE,C34 H32 Fe N4 O4,Cc1c2n3c(c1CCC(=O)O)C=C4C(=C(C5=[N]4[Fe]36[N]7...,616.499,4.73854,2,6


## 9) Fix the protein structure with PDBFixer (missing residues/atoms + add H)

This keeps ligands/ions, and can optionally remove only waters.

In [17]:
if COORD_FORMAT != "pdb":
    raise RuntimeError("PDBFixer section expects a legacy PDB file. Change PDB_ID to one with a legacy PDB, or convert mmCIF->PDB externally.")

def fix_pdb_with_pdbfixer(
    pdb_path: Path,
    ph: float = 7.0,
    build_missing_residues: bool = True,
    remove_waters: bool = True,
) -> app.Modeller:
    fixer = PDBFixer(filename=str(pdb_path))

    fixer.findMissingResidues()
    if not build_missing_residues:
        fixer.missingResidues = {}

    fixer.findNonstandardResidues()
    if fixer.nonstandardResidues:
        fixer.replaceNonstandardResidues()

    fixer.findMissingAtoms()
    fixer.addMissingAtoms()

    fixer.addMissingHydrogens(ph)

    modeller = app.Modeller(fixer.topology, fixer.positions)
    if remove_waters:
        modeller.deleteWater()
    return modeller

modeller = fix_pdb_with_pdbfixer(
    RAW_PDB_PATH,
    ph=PH,
    build_missing_residues=BUILD_MISSING_RESIDUES,
    remove_waters=REMOVE_WATERS,
)

with open(FIXED_PDB_PATH, "w") as f:
    app.PDBFile.writeFile(modeller.topology, modeller.positions, f, keepIds=True)

print("Saved fixed PDB:", FIXED_PDB_PATH)


Saved fixed PDB: /Users/ASUS/Downloads/4HHB_workflow/4HHB_fixed.pdb


## 10) Extract ligand instances from the fixed PDB and save ligand-only PDB

In [18]:
fixed_lines = FIXED_PDB_PATH.read_text(errors="ignore").splitlines()

def find_ligand_instances(pdb_lines: Iterable[str], lig_id: str) -> List[Tuple[str, int, str]]:
    lig_id = lig_id.upper()
    instances = set()
    for line in pdb_lines:
        if line.startswith("HETATM") and line[17:20].strip().upper() == lig_id:
            chain = line[21].strip()
            resseq = int(line[22:26])
            icode = line[26].strip()
            instances.add((chain, resseq, icode))
    return sorted(instances)

instances = find_ligand_instances(fixed_lines, LIG_ID)
print("Ligand instances (chain, resseq, icode):")
for t in instances:
    print("  ", t)

def write_ligand_only_pdb(modeller: app.Modeller, ligand_resname: str, out_pdb: Path):
    ligand_resname = ligand_resname.strip().upper()
    lig_res = [res for res in modeller.topology.residues() if res.name.upper() == ligand_resname]
    if not lig_res:
        raise ValueError(f"No residues named '{ligand_resname}' found in the fixed topology.")

    m2 = app.Modeller(modeller.topology, modeller.positions)
    to_delete = [res for res in m2.topology.residues() if res.name.upper() != ligand_resname]
    m2.delete(to_delete)

    with open(out_pdb, "w") as f:
        app.PDBFile.writeFile(m2.topology, m2.positions, f, keepIds=True)

write_ligand_only_pdb(modeller, LIG_ID, LIGAND_ONLY_PDB_PATH)
print("Saved ligand-only PDB:", LIGAND_ONLY_PDB_PATH)


Ligand instances (chain, resseq, icode):
   ('A', 142, '')
   ('B', 148, '')
   ('C', 142, '')
   ('D', 148, '')
Saved ligand-only PDB: /Users/ASUS/Downloads/4HHB_workflow/4HHB_HEM_ligand_only.pdb


## 11) Download ligand files from RCSB

- Ideal CCD CIF: `<LIG_ID>.cif`
- Ligand instance coordinates: `models.rcsb.org` ligand endpoint (first instance by default).

In [19]:
# Ideal CCD CIF
ccd_cif_url = f"{LIGANDS_URL}/{LIG_ID.upper()}.cif"
ccd_cif_path = OUTDIR / f"{LIG_ID.upper()}.cif"
try:
    download_file(ccd_cif_url, ccd_cif_path)
    print("Downloaded CCD CIF:", ccd_cif_path)
except Exception as e:
    print("Could not download CCD CIF:", e)

# Ligand instance coordinates (pick first)
if instances:
    chain, resseq, icode = instances[0]
    encoding = "sdf"  # try "mol" or "mol2" if you prefer
    lig_instance_url = f"{MODELS_URL}/{PDB_ID}/ligand?auth_asym_id={chain}&auth_seq_id={resseq}&encoding={encoding}"
    lig_instance_path = OUTDIR / f"{PDB_ID}_{LIG_ID}_{chain}{resseq}.{encoding}"

    try:
        download_file(lig_instance_url, lig_instance_path)
        print("Downloaded ligand instance coords:", lig_instance_path)
        print("From:", lig_instance_url)
    except Exception as e:
        print("Could not download ligand instance coords:", e)
else:
    print("No ligand instances found; skipping instance download.")


Downloaded CCD CIF: /Users/ASUS/Downloads/4HHB_workflow/HEM.cif
Downloaded ligand instance coords: /Users/ASUS/Downloads/4HHB_workflow/4HHB_HEM_A142.sdf
From: https://models.rcsb.org/v1/4HHB/ligand?auth_asym_id=A&auth_seq_id=142&encoding=sdf


## 12) Visualize fixed protein + ligand in py3Dmol

In [20]:
pdb_text_fixed = FIXED_PDB_PATH.read_text(errors="ignore")

view = py3Dmol.view(width=900, height=550)
view.addModel(pdb_text_fixed, "pdb")
view.setStyle({"protein": True}, {"cartoon": {}})
view.setStyle({"resn": LIG_ID.upper()}, {"stick": {}})
view.zoomTo({"resn": LIG_ID.upper()})
view.show()


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [21]:
# --- Settings (edit) ---
UNIPROT_QUERY = UNIPROT_ACC  # reuse from earlier section (e.g., "P69905"), or set directly
MAX_ENTRIES_TO_TABULATE = 250  # keep this modest for a fast demo; set None to try all (can be slow for very large targets)
BATCH_SIZE = 200  # search pagination batch size (<= 10000 is allowed)

def _search_entries_by_uniprot_raw(uniprot_acc: str, rows: int = 25, start: int = 0) -> dict:
    # Return full JSON response (including total_count) for a UniProt->PDB search.
    uniprot_acc = uniprot_acc.strip()
    if not uniprot_acc:
        return {"total_count": 0, "result_set": []}

    query = {
        "query": {
            "type": "group",
            "logical_operator": "and",
            "nodes": [
                {
                    "type": "terminal",
                    "service": "text",
                    "parameters": {
                        "attribute": "rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_accession",
                        "operator": "exact_match",
                        "value": uniprot_acc,
                    },
                }
            ],
        },
        "request_options": {
            "paginate": {"start": int(start), "rows": int(rows)},
            "results_content_type": ["experimental"],  # experimental PDB structures
        },
        "return_type": "entry",
    }

    r = session.post(SEARCH_URL, json=query, timeout=60)
    if not r.ok:
        raise RuntimeError(f"HTTP {r.status_code}: {r.text}")
    return r.json()

def uniprot_entry_count(uniprot_acc: str) -> int:
    # Get total number of PDB entry hits for a UniProt accession (fast).
    data = _search_entries_by_uniprot_raw(uniprot_acc, rows=1, start=0)
    return int(data.get("total_count", 0))

def fetch_entry_ids_for_uniprot(uniprot_acc: str, max_entries: Optional[int] = None,
                                batch_size: int = 200) -> List[str]:
    # Fetch entry IDs (PDB IDs) for a UniProt accession, with pagination.
    out: List[str] = []
    start = 0
    while True:
        rows = batch_size
        if max_entries is not None:
            remaining = max_entries - len(out)
            if remaining <= 0:
                break
            rows = min(rows, remaining)

        data = _search_entries_by_uniprot_raw(uniprot_acc, rows=rows, start=start)
        ids = [x["identifier"] for x in data.get("result_set", [])]
        if not ids:
            break

        out.extend(ids)
        start += len(ids)

        # If server returned fewer than requested, we're done
        if len(ids) < rows:
            break

    return out

def get_entry_resolution_and_method(pdb_id: str) -> dict:
    # Fetch resolution + experimental method for an entry via the REST Data API core/entry.
    pdb_id = pdb_id.upper()
    j = request_json(f"{DATA_CORE_URL}/entry/{pdb_id}")

    # Resolution is typically a list, e.g. [1.74]
    res_list = (j.get("rcsb_entry_info") or {}).get("resolution_combined") or []
    resolution = res_list[0] if res_list else None

    # Experimental method: exptl is a list of dicts; take first method if present
    exptl = j.get("exptl") or []
    method = exptl[0].get("method") if exptl and isinstance(exptl[0], dict) else None

    release = (j.get("rcsb_accession_info") or {}).get("initial_release_date")
    return {"pdb_id": pdb_id, "resolution_A": resolution, "method": method, "release_date": release}

# --- Run ---
total = uniprot_entry_count(UNIPROT_QUERY)
print(f"Total PDB entries for UniProt {UNIPROT_QUERY}: {total}")

entry_ids = fetch_entry_ids_for_uniprot(
    UNIPROT_QUERY,
    max_entries=MAX_ENTRIES_TO_TABULATE,
    batch_size=BATCH_SIZE,
)
print(f"Fetched {len(entry_ids)} PDB IDs for tabulation.")

rows = []
for pdb_id in tqdm(entry_ids, desc="Fetching entry metadata"):
    try:
        rows.append(get_entry_resolution_and_method(pdb_id))
    except Exception as e:
        rows.append({"pdb_id": pdb_id, "resolution_A": None, "method": None, "release_date": None, "error": str(e)})

df_entries = pd.DataFrame(rows).sort_values(["resolution_A", "pdb_id"], na_position="last").reset_index(drop=True)
display(df_entries.head(25))

# Optional: save
entries_csv = OUTDIR / f"{UNIPROT_QUERY}_pdb_entries_resolution.csv"
df_entries.to_csv(entries_csv, index=False)
print("Saved:", entries_csv)

Total PDB entries for UniProt P69905: 353
Fetched 250 PDB IDs for tabulation.


Fetching entry metadata:   0%|          | 0/250 [00:00<?, ?it/s]

,pdb_id,resolution_A,method,release_date,error
0,2W72,1.070,X-RAY DIFFRACTION,2009-04-28T00:00:00+0000,NaN
1,1IRD,1.250,X-RAY DIFFRACTION,2001-10-17T00:00:00+0000,NaN
2,2DN1,1.250,X-RAY DIFFRACTION,2006-05-09T00:00:00+0000,NaN
3,2DN2,1.250,X-RAY DIFFRACTION,2006-05-09T00:00:00+0000,NaN
4,2DN3,1.250,X-RAY DIFFRACTION,2006-05-09T00:00:00+0000,NaN
5,3S66,1.401,X-RAY DIFFRACTION,2011-08-03T00:00:00+0000,NaN
6,1J40,1.450,X-RAY DIFFRACTION,2003-07-22T00:00:00+0000,NaN
7,1J41,1.450,X-RAY DIFFRACTION,2003-07-22T00:00:00+0000,NaN
8,2D5Z,1.450,X-RAY DIFFRACTION,2006-03-07T00:00:00+0000,NaN
9,1BAB,1.500,X-RAY DIFFRACTION,1994-01-31T00:00:00+0000,NaN


Saved: /Users/ASUS/Downloads/4HHB_workflow/P69905_pdb_entries_resolution.csv


In [22]:
GRAPHQL_URL = "https://data.rcsb.org/graphql"

def graphql(query: str, variables: Optional[dict] = None) -> dict:
    payload = {"query": query, "variables": variables or {}}
    r = session.post(GRAPHQL_URL, json=payload, timeout=60)
    r.raise_for_status()
    data = r.json()
    if "errors" in data and data["errors"]:
        raise RuntimeError(data["errors"][0].get("message", "GraphQL error"))
    return data.get("data", {})

def ligand_metadata_for_entries(entry_ids: List[str]) -> pd.DataFrame:
    # Return ligand metadata for a list of entry IDs (PDB IDs).
    entry_ids = [e.upper() for e in entry_ids]
    q = '''
    query Ligands($ids: [String!]!) {
      entries(entry_ids: $ids) {
        rcsb_id
        nonpolymer_entities {
          nonpolymer_comp {
            chem_comp {
              id
              type
              formula_weight
              name
              formula
            }
          }
        }
      }
    }
    '''
    data = graphql(q, {"ids": entry_ids})
    out_rows = []
    for entry in data.get("entries") or []:
        pdb_id = entry.get("rcsb_id")
        for np in entry.get("nonpolymer_entities") or []:
            cc = (((np or {}).get("nonpolymer_comp") or {}).get("chem_comp") or {})
            if not cc:
                continue
            out_rows.append(
                {
                    "structureId": pdb_id,
                    "chemicalID": cc.get("id"),
                    "type": cc.get("type"),
                    "molecularWeight": cc.get("formula_weight"),
                    "chemicalName": cc.get("name"),
                    "formula": cc.get("formula"),
                }
            )
    df = pd.DataFrame(out_rows)
    if df.empty:
        return df

    # Count repeated occurrences of the same chemical in the same structure
    df["instanceCount"] = 1
    df = (
        df.groupby(["structureId", "chemicalID", "type", "molecularWeight", "chemicalName", "formula"], dropna=False)["instanceCount"]
        .sum()
        .reset_index()
        .sort_values(["structureId", "instanceCount", "chemicalID"], ascending=[True, False, True])
        .reset_index(drop=True)
    )
    return df

# --- Choose top structures by best (lowest) resolution ---
TOP_N_STRUCTURES = 10
df_top = df_entries.dropna(subset=["resolution_A"]).head(TOP_N_STRUCTURES)
top_ids = df_top["pdb_id"].tolist()
print("Top structures:", top_ids)

df_ligands = ligand_metadata_for_entries(top_ids)

# Optional: remove common crystallization components (edit as needed)
EXCLUDE_COMMON = {"HOH"}  # add e.g. {"HOH","SO4","NA","CL","EDO","GOL","PEG","ACT"}
df_ligands_filtered = df_ligands[~df_ligands["chemicalID"].isin(EXCLUDE_COMMON)].reset_index(drop=True)

display(df_ligands_filtered)

# Optional: save
lig_csv = OUTDIR / f"{UNIPROT_QUERY}_top{TOP_N_STRUCTURES}_ligand_metadata.csv"
df_ligands_filtered.to_csv(lig_csv, index=False)
print("Saved:", lig_csv)

Top structures: ['2W72', '1IRD', '2DN1', '2DN2', '2DN3', '3S66', '1J40', '1J41', '2D5Z', '1BAB']


,structureId,chemicalID,type,molecularWeight,chemicalName,formula,instanceCount
0,1BAB,HEM,non-polymer,616.487,PROTOPORPHYRIN IX CONTAINING FE,C34 H32 Fe N4 O4,1
1,1BAB,SO4,non-polymer,96.063,SULFATE ION,O4 S,1
2,1IRD,CMO,non-polymer,28.010,CARBON MONOXIDE,C O,1
3,1IRD,HEM,non-polymer,616.487,PROTOPORPHYRIN IX CONTAINING FE,C34 H32 Fe N4 O4,1
4,1J40,2FU,non-polymer,84.073,BUT-2-ENEDIAL,C4 H4 O2,1
5,1J40,CMO,non-polymer,28.010,CARBON MONOXIDE,C O,1
6,1J40,HEM,non-polymer,616.487,PROTOPORPHYRIN IX CONTAINING FE,C34 H32 Fe N4 O4,1
7,1J40,HNI,non-polymer,619.336,PROTOPORPHYRIN IX CONTAINING NI(II),C34 H32 N4 Ni O4,1
8,1J41,2FU,non-polymer,84.073,BUT-2-ENEDIAL,C4 H4 O2,1
9,1J41,CMO,non-polymer,28.010,CARBON MONOXIDE,C O,1


Saved: /Users/ASUS/Downloads/4HHB_workflow/P69905_top10_ligand_metadata.csv
